In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
text = 'Data scientist'

In [3]:
url = 'https://hh.ru'
page = 0
params = {'L_save_area': 'true',
          'clusters': 'true',
          'enable_snippets': 'true',
          'showClusters': 'true',
          'page': page,
          'text': text
          }
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}

In [4]:
def salary(el):
    
    def int_salary(st):
        return int(''.join([i for i in st if i in '0123456789']))
    
    min_salary, max_salary, currency = None, None, None
    
    if el.find('–') > -1:
        min_sal, arg, max_sal = el.partition('–')
        min_salary = int_salary(min_sal)
        max_salary = int_salary(max_sal)
        
    elif el.startswith('до'):
        max_salary = int_salary(el)
        
    elif el.startswith('от'):
        min_salary = int_salary(el)
    
    currency = el[el.rfind(' ')+1:].rstrip('.')
    
    return min_salary, max_salary, currency

In [5]:
vacancy_ls = []
vacancy_list = True
while vacancy_list:
    response = requests.get(url + '/search/vacancy', params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    vacancy_list = dom.find_all('div', {'class': 'vacancy-serp-item'})
    for vacancy in vacancy_list:        
        vacancy_data = {}        
        vacancy_name = vacancy.find('a', {'class': 'bloko-link'}).text        
        serial_link = vacancy.find('a', {'class': 'bloko-link'})['href']        
        try:
            vacancy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation',
                                                   'class': 'bloko-section-header-3_lite'}).getText()
            min_salary, max_salary, currency = salary(vacancy_salary)
        except:
            min_salary, max_salary, currency = None, None, None
            
        vacancy_data['name'] = vacancy_name
        vacancy_data['link'] = serial_link
        vacancy_data['min_salary'] = min_salary
        vacancy_data['max_salary'] = max_salary
        vacancy_data['currency'] = currency
        
        if vacancy_data not in vacancy_ls:
            vacancy_ls.append(vacancy_data)      
            
    page += 1
    params['page'] = page

In [6]:
len(vacancy_ls)

251

In [7]:
df = pd.DataFrame(vacancy_ls)

In [8]:
df.head()

,name,link,min_salary,max_salary,currency
0,Senior Data Scientist,https://samara.hh.ru/vacancy/44573787?query=Da...,NaN,NaN,None
1,Data Scientist,https://samara.hh.ru/vacancy/44350010?query=Da...,100000.0,250000.0,руб
2,Data Scientist,https://samara.hh.ru/vacancy/44240210?query=Da...,150000.0,250000.0,руб
3,Junior Data Scientist,https://samara.hh.ru/vacancy/44526667?query=Da...,NaN,NaN,None
4,Data Scientist,https://samara.hh.ru/vacancy/44467078?query=Da...,1000.0,1500.0,EUR


In [9]:
df.to_excel('hh.xlsx')

In [10]:
df2 = pd.read_csv('hh.csv')

In [11]:
df2

,Unnamed: 0,name,link,min_salary,max_salary,currency
0,0,Senior Data Scientist,https://samara.hh.ru/vacancy/44573787?query=Da...,NaN,NaN,NaN
1,1,Data Scientist,https://samara.hh.ru/vacancy/44350010?query=Da...,100000.0,250000.0,руб
2,2,Data Scientist,https://samara.hh.ru/vacancy/44240210?query=Da...,150000.0,250000.0,руб
3,3,Junior Data Scientist,https://samara.hh.ru/vacancy/44526667?query=Da...,NaN,NaN,NaN
4,4,Data Scientist,https://samara.hh.ru/vacancy/44467078?query=Da...,1000.0,1500.0,EUR
...,...,...,...,...,...,...
253,253,Python разработчик (django),https://samara.hh.ru/vacancy/43889597?query=Da...,NaN,350000.0,руб
254,254,Программист Python (Middle),https://samara.hh.ru/vacancy/42587352?query=Da...,NaN,250000.0,руб
255,255,IT Product manager,https://samara.hh.ru/vacancy/44148965?query=Da...,150000.0,220000.0,руб
256,256,Аналитик по управлению данными,https://samara.hh.ru/vacancy/44067596?query=Da...,NaN,NaN,NaN
